# Introduction to PID Controllers for Aircraft Autopilots

This notebook provides a step-by-step introduction to Proportional-Integral-Derivative (PID) controllers and how they're used in aircraft autopilot systems.

## Learning Objectives

By the end of this notebook, you should be able to:
1. Understand the basic principles of PID control
2. Explain the purpose and effect of each PID component (P, I, D)
3. Implement a basic PID controller in Python
4. Tune PID parameters for desired response characteristics
5. Apply a PID controller to a simple aircraft control problem

## 1. What is a PID Controller?

A PID controller is a control loop feedback mechanism widely used in industrial control systems and a variety of other applications requiring continuously modulated control. A PID controller continuously calculates an error value as the difference between a desired setpoint and a measured process variable and applies a correction based on proportional, integral, and derivative terms.

### The PID Equation

The PID algorithm consists of three separate parameters: the Proportional, Integral, and Derivative values. The weighted sum of these three actions is used to adjust the process via a control element such as an elevator in an aircraft.

The mathematical form of a PID controller is:

$$u(t) = K_p e(t) + K_i \int_{0}^{t} e(\tau) d\tau + K_d \frac{de(t)}{dt}$$

Where:
- $u(t)$ is the control signal
- $e(t)$ is the error value (setpoint - measured value)
- $K_p$, $K_i$, and $K_d$ are the coefficients for the proportional, integral, and derivative terms

Let's implement a basic PID controller and visualize how each component affects the output.

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import time
from IPython.display import clear_output

# Make plots interactive
%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')

In [ ]:
# Import the PID controller from simFlight library
from simflight.controllers import PIDController

## 2. Understanding Each Component of PID

### The Proportional Term (P)

The proportional term produces an output value that is proportional to the current error value. The proportional response can be adjusted by multiplying the error by a constant $K_p$, called the proportional gain.

Let's create a simple simulation to see how a purely proportional controller behaves.

In [ ]:
def simulate_system(controller, setpoint, initial_value, disturbances=None, steps=100, dt=0.1):
    """Simulate a simple first-order system with a PID controller."""
    # Time vector
    time = np.arange(0, steps * dt, dt)
    
    # Initialize arrays to store results
    value = np.zeros(steps)
    value[0] = initial_value
    control_signal = np.zeros(steps)
    p_term = np.zeros(steps)
    i_term = np.zeros(steps)
    d_term = np.zeros(steps)
    
    # System time constant
    tau = 1.0
    
    # Set controller setpoint
    controller.set_setpoint(setpoint)
    
    # Simulate the system
    for i in range(1, steps):
        # Compute control signal
        control_signal[i-1] = controller.compute(value[i-1])
        
        # Record PID terms
        p_term[i-1] = controller.history['p_term'][-1]
        i_term[i-1] = controller.history['i_term'][-1]
        d_term[i-1] = controller.history['d_term'][-1]
        
        # Apply disturbance if specified
        disturbance = 0
        if disturbances is not None:
            for t, mag in disturbances:
                if abs(time[i] - t) < dt:
                    disturbance = mag
        
        # Update system (first-order response)
        value[i] = value[i-1] + (dt/tau) * (control_signal[i-1] - value[i-1]) + disturbance
    
    # Get final control signal
    control_signal[-1] = controller.compute(value[-1])
    p_term[-1] = controller.history['p_term'][-1]
    i_term[-1] = controller.history['i_term'][-1]
    d_term[-1] = controller.history['d_term'][-1]
    
    return time, value, control_signal, p_term, i_term, d_term

def plot_results(time, value, setpoint, control_signal, p_term, i_term, d_term, title):
    """Plot the simulation results."""
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)
    
    # Plot system response
    ax1.plot(time, value, 'b-', linewidth=2, label='System Output')
    ax1.plot(time, [setpoint] * len(time), 'r--', linewidth=2, label='Setpoint')
    ax1.set_ylabel('Value')
    ax1.set_title(title)
    ax1.legend()
    ax1.grid(True)
    
    # Plot control signal and PID terms
    ax2.plot(time, control_signal, 'k-', linewidth=2, label='Control Signal')
    ax2.plot(time, p_term, 'r-', linewidth=1, label='P Term')
    ax2.plot(time, i_term, 'g-', linewidth=1, label='I Term')
    ax2.plot(time, d_term, 'b-', linewidth=1, label='D Term')
    ax2.set_xlabel('Time (s)')
    ax2.set_ylabel('Signal')
    ax2.legend()
    ax2.grid(True)
    
    plt.tight_layout()
    return fig

In [ ]:
# Create a P-only controller
p_controller = PIDController(kp=1.0, ki=0.0, kd=0.0, dt=0.1)

# Simulate the system
setpoint = 100.0
initial_value = 0.0
time, value, control_signal, p_term, i_term, d_term = simulate_system(
    p_controller, setpoint, initial_value, steps=200)

# Plot the results
plot_results(time, value, setpoint, control_signal, p_term, i_term, d_term, 
             'P Controller Response (Kp=1.0)')

### Observations about the P Controller

- The proportional controller responds immediately to error
- As the system output approaches the setpoint, the error decreases, so the control signal decreases
- Notice the **steady-state error**: the system never quite reaches the setpoint because as it gets closer, the P term gets smaller
- This is a characteristic limitation of a purely proportional controller

Let's try increasing the P gain to see how it affects the response:

In [ ]:
# Create a P-only controller with higher gain
p_controller_high = PIDController(kp=5.0, ki=0.0, kd=0.0, dt=0.1)

# Simulate the system
time, value, control_signal, p_term, i_term, d_term = simulate_system(
    p_controller_high, setpoint, initial_value, steps=200)

# Plot the results
plot_results(time, value, setpoint, control_signal, p_term, i_term, d_term, 
             'P Controller Response (Kp=5.0)')

### The Integral Term (I)

The integral term sums up the error over time. If there is a residual steady-state error, the integral term will increase over time to compensate for it. This helps eliminate the steady-state error we observed with the P-only controller.

Let's see how a PI controller behaves:

In [ ]:
# Create a PI controller
pi_controller = PIDController(kp=1.0, ki=0.1, kd=0.0, dt=0.1)

# Simulate the system
time, value, control_signal, p_term, i_term, d_term = simulate_system(
    pi_controller, setpoint, initial_value, steps=200)

# Plot the results
plot_results(time, value, setpoint, control_signal, p_term, i_term, d_term, 
             'PI Controller Response (Kp=1.0, Ki=0.1)')

### Observations about the PI Controller

- The proportional term still provides immediate response to error
- The integral term gradually increases to compensate for the steady-state error
- Notice how the system now reaches the setpoint, eliminating the steady-state error
- However, the response might exhibit some overshoot due to the integral term continuing to accumulate

### The Derivative Term (D)

The derivative term predicts system behavior based on the rate of change of the error. It helps dampen the system response and reduce overshoot. It can be thought of as applying the brakes as we approach the setpoint.

Let's see how a complete PID controller behaves:

In [ ]:
# Create a PID controller
pid_controller = PIDController(kp=1.0, ki=0.1, kd=0.5, dt=0.1)

# Simulate the system
time, value, control_signal, p_term, i_term, d_term = simulate_system(
    pid_controller, setpoint, initial_value, steps=200)

# Plot the results
plot_results(time, value, setpoint, control_signal, p_term, i_term, d_term, 
             'PID Controller Response (Kp=1.0, Ki=0.1, Kd=0.5)')

### Observations about the PID Controller

- The derivative term helps predict where the system is heading and applies the brakes to prevent overshoot
- Notice how the D term is large at the beginning (when error is changing rapidly) and diminishes as the system settles
- The system response is now faster and more stable, with less overshoot than the PI controller

## 3. Responding to Disturbances

One of the key benefits of a PID controller is its ability to respond to disturbances. Let's see how our different controllers handle disturbances:

In [ ]:
# Define disturbances at specific times
disturbances = [(10.0, 20.0), (15.0, -15.0)]

# Create controllers
p_controller = PIDController(kp=1.0, ki=0.0, kd=0.0, dt=0.1)
pi_controller = PIDController(kp=1.0, ki=0.1, kd=0.0, dt=0.1)
pid_controller = PIDController(kp=1.0, ki=0.1, kd=0.5, dt=0.1)

# Simulate with disturbances
time_p, value_p, control_p, p_term_p, i_term_p, d_term_p = simulate_system(
    p_controller, setpoint, initial_value, disturbances, steps=300)

time_pi, value_pi, control_pi, p_term_pi, i_term_pi, d_term_pi = simulate_system(
    pi_controller, setpoint, initial_value, disturbances, steps=300)

time_pid, value_pid, control_pid, p_term_pid, i_term_pid, d_term_pid = simulate_system(
    pid_controller, setpoint, initial_value, disturbances, steps=300)

# Plot comparative results
plt.figure(figsize=(12, 8))
plt.plot(time_p, value_p, 'r-', linewidth=2, label='P Controller')
plt.plot(time_pi, value_pi, 'g-', linewidth=2, label='PI Controller')
plt.plot(time_pid, value_pid, 'b-', linewidth=2, label='PID Controller')
plt.plot(time_p, [setpoint] * len(time_p), 'k--', linewidth=1, label='Setpoint')

# Mark disturbances
for t, mag in disturbances:
    plt.axvline(x=t, color='gray', linestyle='--', alpha=0.5)
    plt.text(t+0.1, setpoint+10, f'Disturbance: {mag}', fontsize=10)

plt.xlabel('Time (s)')
plt.ylabel('Value')
plt.title('Controller Responses to Disturbances')
plt.legend()
plt.grid(True)
plt.tight_layout()

### Observations about Disturbance Handling

- The P controller responds to disturbances but maintains a steady-state error
- The PI controller eventually returns to the setpoint after disturbances, but may take longer
- The PID controller provides the best overall response to disturbances, with quick recovery and minimal overshoot

## 4. PID Controller Tuning

Tuning a PID controller involves finding the right values for Kp, Ki, and Kd to achieve the desired system response. Let's create an interactive tool to experiment with different PID parameter values:

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Create sliders for PID parameters
kp_slider = widgets.FloatSlider(value=1.0, min=0.0, max=10.0, step=0.1, description='Kp:')
ki_slider = widgets.FloatSlider(value=0.1, min=0.0, max=2.0, step=0.01, description='Ki:')
kd_slider = widgets.FloatSlider(value=0.5, min=0.0, max=5.0, step=0.1, description='Kd:')

# Create output widget
output = widgets.Output()

# Function to update plot
def update_plot(kp, ki, kd):
    with output:
        clear_output(wait=True)
        
        # Create PID controller with current parameters
        controller = PIDController(kp=kp, ki=ki, kd=kd, dt=0.1)
        
        # Simulate the system
        time, value, control_signal, p_term, i_term, d_term = simulate_system(
            controller, setpoint, initial_value, steps=200)
        
        # Plot the results
        plot_results(time, value, setpoint, control_signal, p_term, i_term, d_term,
                     f'PID Controller Response (Kp={kp:.1f}, Ki={ki:.2f}, Kd={kd:.1f})')
        plt.show()

# Connect the sliders to the update function
widgets.interactive_output(update_plot, {'kp': kp_slider, 'ki': ki_slider, 'kd': kd_slider})

# Display the sliders and output
display(widgets.VBox([kp_slider, ki_slider, kd_slider, output]))
update_plot(1.0, 0.1, 0.5)

### PID Tuning Guidelines

1. **Start with P only**: Set Ki and Kd to zero and increase Kp until you get a reasonable response time, but without too much overshoot.

2. **Add I to eliminate steady-state error**: Gradually increase Ki until the steady-state error is eliminated in a reasonable time, without causing too much overshoot.

3. **Add D to reduce overshoot**: Gradually increase Kd to reduce overshoot and dampen the response, without making the system too sluggish.

4. **Fine-tune all parameters**: Make small adjustments to all three parameters to optimize the response.

## 5. Applying PID to Aircraft Control

Now that we understand the basics of PID control, let's see how it can be applied to aircraft autopilot systems. We'll use the simFlight library to simulate an altitude hold autopilot.

In [ ]:
from simflight.simulators import VirtualSimulator
from simflight.aircraft import Aircraft
from simflight.autopilot import AltitudeHold
from simflight.visualization import plot_simulation_results, plot_pid_performance

In [ ]:
# Create a virtual simulator and aircraft
simulator = VirtualSimulator()
aircraft = Aircraft(simulator)

# Initialize the aircraft state
aircraft.reset(altitude=5000.0, airspeed=150.0, heading=0.0)

# Create an altitude hold autopilot
altitude_hold = AltitudeHold(kp=0.01, ki=0.001, kd=0.05)

# Set the target altitude
target_altitude = 8000.0  # feet
altitude_hold.set_target(target_altitude)
altitude_hold.enable()

# Run the simulation
simulation_time = 120  # seconds
dt = 0.1  # seconds
steps = int(simulation_time / dt)

# Record data
time_points = []
state_history = {'altitude': [], 'pitch': [], 'vertical_speed': []}
control_history = {'elevator': []}

# Simulation loop
for i in range(steps):
    current_time = i * dt
    time_points.append(current_time)
    
    # Get current aircraft state
    state = aircraft.get_state()
    
    # Record state
    for key in state_history:
        state_history[key].append(state[key])
    
    # Compute control command
    elevator_command = altitude_hold.compute(state)
    
    # Record control
    control_history['elevator'].append(elevator_command['elevator'])
    
    # Apply control to aircraft
    aircraft.set_controls(elevator=elevator_command['elevator'])
    
    # Advance simulation
    simulator.step(dt)

# Plot results
plot_simulation_results(
    time_points, 
    state_history, 
    setpoints={'altitude': target_altitude},
    control_history=control_history,
    variables=['altitude', 'pitch', 'vertical_speed']
)

# Plot PID performance
plot_pid_performance(altitude_hold.controller.history)

## 6. Interactive Aircraft Altitude Control

Let's create an interactive simulation where you can adjust the PID parameters for the altitude hold autopilot and observe the effects on aircraft behavior:

In [ ]:
# Create widgets for altitude hold parameters
alt_kp_slider = widgets.FloatSlider(value=0.01, min=0.001, max=0.05, step=0.001, description='Alt Kp:')
alt_ki_slider = widgets.FloatSlider(value=0.001, min=0.0, max=0.01, step=0.0001, description='Alt Ki:')
alt_kd_slider = widgets.FloatSlider(value=0.05, min=0.0, max=0.2, step=0.01, description='Alt Kd:')
target_alt_slider = widgets.FloatSlider(value=8000, min=1000, max=20000, step=500, description='Target Alt:')

# Create output widget
output = widgets.Output()

# Function to run simulation with current parameters
def run_altitude_simulation(alt_kp, alt_ki, alt_kd, target_alt):
    with output:
        clear_output(wait=True)
        
        # Create simulator and aircraft
        simulator = VirtualSimulator()
        aircraft = Aircraft(simulator)
        aircraft.reset(altitude=5000.0, airspeed=150.0, heading=0.0)
        
        # Create altitude hold autopilot with current parameters
        altitude_hold = AltitudeHold(kp=alt_kp, ki=alt_ki, kd=alt_kd)
        altitude_hold.set_target(target_alt)
        altitude_hold.enable()
        
        # Run simulation
        simulation_time = 120  # seconds
        dt = 0.1  # seconds
        steps = int(simulation_time / dt)
        
        # Record data
        time_points = []
        state_history = {'altitude': [], 'pitch': [], 'vertical_speed': []}
        control_history = {'elevator': []}
        
        # Simulation loop
        for i in range(steps):
            current_time = i * dt
            time_points.append(current_time)
            
            # Get current aircraft state
            state = aircraft.get_state()
            
            # Record state
            for key in state_history:
                state_history[key].append(state[key])
            
            # Compute control command
            elevator_command = altitude_hold.compute(state)
            
            # Record control
            control_history['elevator'].append(elevator_command['elevator'])
            
            # Apply control to aircraft
            aircraft.set_controls(elevator=elevator_command['elevator'])
            
            # Advance simulation
            simulator.step(dt)
        
        # Plot results
        fig1 = plot_simulation_results(
            time_points, 
            state_history, 
            setpoints={'altitude': target_alt},
            control_history=control_history,
            variables=['altitude', 'pitch', 'vertical_speed']
        )
        plt.show()
        
        fig2 = plot_pid_performance(altitude_hold.controller.history)
        plt.show()

# Connect widgets to function
widgets.interactive_output(run_altitude_simulation, {
    'alt_kp': alt_kp_slider, 
    'alt_ki': alt_ki_slider, 
    'alt_kd': alt_kd_slider,
    'target_alt': target_alt_slider
})

# Display widgets
display(widgets.VBox([alt_kp_slider, alt_ki_slider, alt_kd_slider, target_alt_slider, output]))
run_altitude_simulation(0.01, 0.001, 0.05, 8000)

## 7. Summary and Next Steps

In this notebook, we've covered the fundamentals of PID controllers and their application to aircraft autopilot systems. Here's a summary of what we've learned:

1. **PID Components**:
   - **P (Proportional)**: Provides immediate response proportional to the current error
   - **I (Integral)**: Eliminates steady-state error by accumulating past errors
   - **D (Derivative)**: Improves stability by dampening based on the rate of change of error

2. **Tuning Process**:
   - Start with P only, then add I, then add D
   - Make small adjustments and observe the system response
   - Consider the tradeoffs between speed, stability, and overshoot

3. **Aircraft Control Applications**:
   - Altitude control using elevator
   - Response characteristics depend on PID parameters
   - Different flight phases may require different tuning

### Next Steps

To continue your learning, you might want to explore:

1. **Multiple Control Loops**: Combining altitude, heading, and speed control
2. **Advanced Control Techniques**: Gain scheduling, cascade control, model predictive control
3. **Real-World Applications**: Connect to X-Plane for more realistic simulation
4. **System Identification**: Determining the dynamic model of an aircraft from flight data

Check out the other notebooks in this series for more advanced topics!